In [1]:
# Libraries
import pandas as pd
import numpy as np
import vcf
#
COLORS = {
    "Artery-Aorta":"salmon",
    "Artery-Tibial": "red",
    "Adipose-Subcutaneous": "darkorange",    
    "Adipose-Visceral(Omentum)":"orange",
    "Brain-Caudate(basalganglia)":"lemonchiffon"   , 
    "Brain-Cerebellum":"yellow",
    "Cells-Transformedfibroblasts": "skyblue",
    "Esophagus-Mucosa": "sienna",
    "Esophagus-Muscularis":"burlywood",
    "Heart-LeftVentricle":"darkviolet",
    "Lung": "greenyellow",
    "Muscle-Skeletal": "mediumslateblue",
    "Nerve-Tibial":"gold",
    "Skin-NotSunExposed(Suprapubic)":"blue",
    "Skin-SunExposed(Lowerleg)":"cornflowerblue",
    "Thyroid":"green",
    "WholeBlood": "m",
    "permuted": "gray"
}

#    "Thyroid": "green",
SHORTEN = {
    "Artery-Aorta":"Artery.A"     ,
    "Artery-Tibial": "Artery.T",
    "Adipose-Subcutaneous": "Adipose.S",    
    "Adipose-Visceral(Omentum)":"Adipose.V",
    "Brain-Caudate(basalganglia)":"Caudate"   , 
    "Brain-Cerebellum":"Cerebellum",
    "Cells-Transformedfibroblasts": "Fibroblast",
    "Esophagus-Mucosa": "Mucosa",
    "Esophagus-Muscularis":"Muscularis",
    "Heart-LeftVentricle":"Ventricule",
    "Lung": "Lung",
    "Muscle-Skeletal": "Muscle",
    "Nerve-Tibial":"Nerve",
    "Skin-NotSunExposed(Suprapubic)": "SkinUnexposed",
    "Skin-SunExposed(Lowerleg)":"SkinLeg",
    "Thyroid":"Thyroid",
    "WholeBlood": "Blood",
    "permuted":"Permuted",
    "LCL": "LCL"
}

GENE = '/storage/szfeupe/Runs/GTEx_estr/FEATURES/Genes_only_table'
PATH= "/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/"
TISSUES = sorted([item for item in list(COLORS.keys()) if item != "permuted"])
motif='/storage/resources/dbase/human/hg19/hg19.hipstr_reference_withmotif.bed'

In [2]:
#Linear regression QVALUES
#
GENES = pd.read_csv(GENE, sep='\t')
col=['chrom','str.start','str.id','gene']
SUMMARY = pd.read_csv(PATH +'WholeBlood/PQValues', sep = '\t')[['chrom','str.start','str.id','gene']]
estrs = []
SUM = {}

for T in TISSUES:
    table = pd.read_csv(PATH +T+'/PQValues', sep = '\t')
    table['eSTR.'+SHORTEN[T]] = np.where(table['qvalue']<=0.1, 1, 0)  #10%fdr
    table['qval.'+SHORTEN[T]] = table['qvalue']
    table['beta.'+SHORTEN[T]] = table['beta']
    table = table[col + ['eSTR.'+SHORTEN[T],'qval.'+SHORTEN[T],'beta.'+SHORTEN[T]]]
    table['str.start'].astype(int)
    table['chrom'].astype(str)
    estrs.append('eSTR.'+SHORTEN[T])
    SUM[T]=table
    table['eSTR.'+SHORTEN[T]] = table['eSTR.'+SHORTEN[T]].astype(int)
    SUMMARY = pd.merge(SUMMARY,table[col], on=col, how='right').drop_duplicates().reset_index(drop=True)
    #print(table.shape[0],'\t',table.loc[table['eSTR.'+SHORTEN[T]]==1].shape[0],'\t', T)

SUMMARY = SUMMARY.sort_values(['chrom','gene']).reset_index(drop=True)

print('\n\t',len(SUM.keys()),'\n')
SUMMARY['E.tissues']=[0]*SUMMARY.shape[0]
for T in TISSUES :
    SUMMARY = pd.merge(SUMMARY.fillna(0), SUM[T], how='outer',on=col)
    SUMMARY['E.tissues'] = SUMMARY['E.tissues'].fillna(0) + SUMMARY['eSTR.'+SHORTEN[T]]
    print(SUMMARY.shape[0],'\t',SUMMARY.loc[SUMMARY['eSTR.'+SHORTEN[T]]==1].shape[0],'\t', T)
print('END')
print('Number of egenes... ',SUMMARY.loc[SUMMARY['E.tissues']>=1].shape)
print('Nbr of eGenes in more than 1 tissues ',SUMMARY.loc[SUMMARY['E.tissues']>=2].shape)
print('egenes in at least 10 tissues ',SUMMARY.loc[SUMMARY['E.tissues']>=10].shape)

Y=pd.read_csv(motif, sep='\t', header=None)
Y.columns=['chrom','str.start','str.end','period','motif']
print(Y.shape)
Z = pd.merge(SUMMARY, Y, on=['chrom','str.start'], how='inner')

Z.to_csv('/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/LR_SummaryTest_Table.tsv', sep='\t', index=False)



	 17 

17549 	 1353 	 Adipose-Subcutaneous
21494 	 693 	 Adipose-Visceral(Omentum)
24369 	 821 	 Artery-Aorta
26494 	 1257 	 Artery-Tibial
28860 	 204 	 Brain-Caudate(basalganglia)
30634 	 498 	 Brain-Cerebellum
31852 	 1108 	 Cells-Transformedfibroblasts
33032 	 1228 	 Esophagus-Mucosa
33977 	 1028 	 Esophagus-Muscularis
34812 	 605 	 Heart-LeftVentricle
35582 	 1065 	 Lung
36227 	 1273 	 Muscle-Skeletal
36757 	 1681 	 Nerve-Tibial
37417 	 840 	 Skin-NotSunExposed(Suprapubic)
37917 	 1462 	 Skin-SunExposed(Lowerleg)
38365 	 1523 	 Thyroid
38365 	 883 	 WholeBlood
END
Number of egenes...  (2436, 56)
Nbr of eGenes in more than 1 tissues  (1029, 56)
egenes in at least 10 tissues  (149, 56)
(1620037, 5)


In [7]:
Z[Z['E.tissues']>=1]

,chrom,str.start,str.id,gene,E.tissues,eSTR.Adipose.S,qval.Adipose.S,beta.Adipose.S,eSTR.Adipose.V,qval.Adipose.V,...,beta.SkinLeg,eSTR.Thyroid,qval.Thyroid,beta.Thyroid,eSTR.Blood,qval.Blood,beta.Blood,str.end,period,motif
5,chr1,169667612,STR_169667612,ENSG00000171806.7,10.0,1.0,0.013119,0.225821,1.0,0.012628,...,0.271168,1.0,0.002411,0.300545,0.0,0.120875,0.156997,169667695,2,AC
13,chr1,24722732,STR_24722732,ENSG00000001461.12,5.0,1.0,0.002911,-0.247463,0.0,0.172164,...,-0.083103,1.0,0.005035,0.225859,0.0,0.753668,0.057675,24722801,4,AGAT
15,chr1,33570145,STR_33570145,ENSG00000004455.12,1.0,0.0,1.000000,-0.023106,0.0,0.276575,...,-0.126850,0.0,0.775774,-0.054685,0.0,0.989245,-0.029877,33570159,2,AT
16,chr1,33570145,STR_33570145,ENSG00000142920.12,3.0,1.0,0.076144,0.168071,0.0,1.000000,...,0.099067,0.0,0.941838,0.032911,0.0,0.906231,0.040302,33570159,2,AT
26,chr1,113008068,STR_113008068,ENSG00000007341.14,2.0,0.0,0.116952,-0.168660,1.0,0.037726,...,-0.017639,1.0,0.045158,-0.196958,0.0,1.000000,-0.024674,113008089,2,AC
27,chr1,113008068,STR_113008068,ENSG00000134245.13,1.0,1.0,0.047470,-0.218261,0.0,0.000000,...,0.000000,1.0,0.067806,-0.208491,0.0,0.424944,-0.135200,113008089,2,AC
52,chr1,171273915,STR_171273915,ENSG00000076258.5,7.0,0.0,0.275094,-0.122807,0.0,0.520554,...,-0.220408,0.0,0.832598,0.047229,0.0,0.744333,0.060853,171273962,3,AGG
54,chr1,24056743,STR_24056743,ENSG00000011007.8,1.0,0.0,0.788348,0.055604,0.0,1.000000,...,0.007293,0.0,1.000000,-0.012909,1.0,0.004127,-0.258799,24056790,3,AAT
85,chr1,32621305,STR_32621305,ENSG00000025800.9,2.0,0.0,0.223601,-0.132298,0.0,1.000000,...,-0.093618,0.0,0.923425,0.035571,0.0,0.157014,-0.137807,32621345,4,AAAT
86,chr1,32621305,STR_32621305,ENSG00000084623.7,2.0,0.0,0.792925,0.053931,0.0,0.402906,...,0.103535,0.0,0.610993,0.075929,0.0,0.611502,0.076226,32621345,4,AAAT


In [18]:
Y=pd.read_csv(motif, sep='\t', header=None)
Y.columns=['chrom','str.start','str.end','period','motif']
print(T.shape)
Z = pd.merge(T, Y, on=['chrom','str.start'], how='inner')
print(Z.shape)
Z

(38365, 5)
(38365, 8)


,chrom,str.start,str.id,gene,E.tissues,str.end,period,motif
0,chr1,169895824,STR_169895824,ENSG00000000457.9,0.0,169895846,2,AC
1,chr1,169895824,STR_169895824,ENSG00000000460.12,0.0,169895846,2,AC
2,chr1,169895824,STR_169895824,ENSG00000075945.8,0.0,169895846,2,AC
3,chr1,169667612,STR_169667612,ENSG00000000460.12,0.0,169667695,2,AC
4,chr1,169667612,STR_169667612,ENSG00000007908.11,0.0,169667695,2,AC
5,chr1,169667612,STR_169667612,ENSG00000171806.7,10.0,169667695,2,AC
6,chr1,169667612,STR_169667612,ENSG00000188404.4,0.0,169667695,2,AC
7,chr1,169667612,STR_169667612,ENSG00000174175.12,0.0,169667695,2,AC
8,chr1,27923888,STR_27923888,ENSG00000000938.8,0.0,27923907,2,AG
9,chr1,27923888,STR_27923888,ENSG00000126705.9,0.0,27923907,2,AG


In [12]:
SUMMARY.loc[SUMMARY['gene']=='ENSG00000000460.12']

,chrom,str.start,str.id,gene,E.tissues,eSTR.Adipose.S,qval.Adipose.S,beta.Adipose.S,eSTR.Adipose.V,qval.Adipose.V,...,beta.SkinUnexposed,eSTR.SkinLeg,qval.SkinLeg,beta.SkinLeg,eSTR.Thyroid,qval.Thyroid,beta.Thyroid,eSTR.Blood,qval.Blood,beta.Blood
1,chr1,169667612,STR_169667612,ENSG00000000460.12,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.1606,0.179192,0.0,0.428696,0.134485
11572,chr1,169614029,STR_169614029,ENSG00000000460.12,NaN,0.0,1.0,0.040812,0.0,1.0,...,0.100641,0.0,0.000000,0.000000,0.0,0.0000,0.000000,NaN,NaN,NaN
26494,chr1,169895824,STR_169895824,ENSG00000000460.12,NaN,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.607456,0.110049,0.0,0.0000,0.000000,NaN,NaN,NaN


In [ ]:
see=0 #Add the Alleles 
see=pd.read_csv('/storage/szfeupe/Runs/GTEx_estr/STR_Locus_Alleles.tsv', sep='\t')#q, header=None)
#see.columns=['chrom','str.start', "allele.REF",'allele.ALT','motif_len','ref_counts']
see['str.id']='str_'+see['str.start'].astype(str)
C=['chrom','str.start','str.id',"allele.REF",'allele.ALT','motif_len','ref_counts']
See=see[C]
del See['str.id']
cols = ['chrom','str.start']
OUTs = pd.merge(OUT, See, how='left',on=cols)
del OUTs['ID']
OUTs.shape

In [ ]:
cols=['chrom','str.start','str.id','gene','ID']
#Merge to add beta and pvalues by tissue
OUT1 = pd.merge(OUTs, wb, how='left',on=cols[:4])
OUT2 = pd.merge(OUT1, ads, how='left',on=cols) ; OUT1=0
OUT1 = pd.merge(OUT2, ctf, how='left',on=cols) ; OUT2=0
OUT2 = pd.merge(OUT1, esm, how='left',on=cols) ; OUT1=0
OUT1 = pd.merge(OUT2, ms, how='left',on=cols)  ; OUT2=0
OUT2 = pd.merge(OUT1, lng, how='left',on=cols) ; OUT1=0
OUT1 = pd.merge(OUT2, art, how='left',on=cols)
OUT1.shape

In [ ]:
#Merge to add qvalues and eSTRs by tissue
OUT3 = pd.merge(OUT1, wbp, how='left',on=cols)
OUT2 = pd.merge(OUT3, adsp, how='left',on=cols) ; OUT3=0
OUT3 = pd.merge(OUT2, ctfp, how='left',on=cols) ; OUT2=0
OUT2 = pd.merge(OUT3, esmp, how='left',on=cols) ; OUT3=0
OUT3 = pd.merge(OUT2, msp, how='left',on=cols)  ; OUT2=0
OUT2 = pd.merge(OUT3, lngp, how='left',on=cols) ; OUT3=0
OUT3 = pd.merge(OUT2, artp, how='left',on=cols)
OUT3.shape

In [ ]:
#Label estrs and non estrs by tissue
OUT3['Blood_eSTR?'][OUT3['Blood_eSTR?']!=1] = 0
OUT3['AdipSub_eSTR?'][OUT3['AdipSub_eSTR?']!=1] = 0
OUT3['ArteryT_eSTR?'][OUT3['ArteryT_eSTR?']!=1] = 0
OUT3['Esophagus_eSTR?'][OUT3['Esophagus_eSTR?']!=1] = 0
OUT3['Fibroblast_eSTR?'][OUT3['Fibroblast_eSTR?']!=1] = 0
OUT3['Lung_eSTR?'][OUT3['Lung_eSTR?']!=1] = 0
OUT3['MuscleS_eSTR?'][OUT3['MuscleS_eSTR?']!=1] = 0
OUT3['eSTR_In_tissues'] = OUT3['Blood_eSTR?']+OUT3['Fibroblast_eSTR?']+OUT3['Lung_eSTR?']+OUT3['MuscleS_eSTR?']+OUT3['AdipSub_eSTR?']+OUT3['ArteryT_eSTR?']+OUT3['Esophagus_eSTR?'] 
OUT3.to_csv('/storage/szfeupe/Runs/GTEx_estr/Analysis_by_Tissue/SUMMARY/LR_SummaryTest_Table.tsv', sep='\t', index=False)
#& (OUT3['Blood_eSTR?']==1)& (OUT3['MuscleS_eSTR?']==1)]

In [ ]:
#OUT3['Estrs']=OUT3['Fibroblast_eSTR?']+OUT3['Blood_eSTR?']+OUT3['MuscleS_eSTR?']+OUT3['Lung_eSTR?']+OUT3['Esophagus_eSTR?']+OUT3['ArteryT_eSTR?']+OUT3['AdipSub_eSTR?']
Gene_table = pd.read_csv('/storage/szfeupe/Runs/GTEx_estr/FEATURES/Genes_only_table', sep='\t')
OUT = pd.merge(OUT3, Gene_table[["gene","chrom","gene.name"]], on=["chrom","gene"])
#OUT3=OUT
OUT4=OUT.loc[OUT['eSTR_In_tissues']>=1]
OUT4.to_csv('/storage/szfeupe/Runs/GTEx_estr/eSTRs_Tests_Table.tsv', index=False)

OUT.to_csv('/storage/szfeupe/Runs/GTEx_estr/LR_SummaryTest_Table.tsv', sep='\t', index=False)
print('PP')
print(OUT4.shape)
OUT3.loc[(OUT3['eSTR_In_tissues']>=7)]

In [ ]:
OUT.loc[OUT["eSTR_In_tissues"]>=1].shape

In [ ]:
#(2) in permuted
idx=list(LR.index)
#LRp1 = LRp.loc[LRp['gene'].isin(list(LR['gene'])) & LRp['str.id'].isin(list(LR['str.id']))]
LRp1=LRp.loc[idx]

#select qqplot axis
X= LR['logpval']
Y= LRp1['logpval']

plt.figure(1)
#plot qqplot
sm.qqplot(X, line='45')
sm.qqplot(Y, line='45')
smg.gofplots.qqplot_2samples(X,Y, line='q', xlabel='Observed (log(pval))', ylabel='Permuted (log(pval))')
pylab.show()
#
plt.scatter(X, Y)
plt.ylabel('Observed')
plt.xlabel('Permuted')
plt.show()


In [ ]:
##This code summarises the Anova results and prepare file for anovapvalue qqplot 
##
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.stats as ss

Tissue="WholeBlood"
##delta_bic = snp_bic - snpstr_bic        ##delta_aic = snp_aic - snpstr_aic 
##              the lower tht aic or bic the closer the data is to the model
print('Summary for '+Tissue+' ...\n')

#FDR 
pval = pd.read_csv("/storage/szfeupe/Runs/GTEx_estr/Analysis_by_Tissue/"+Tissue+"/PQValues.txt", sep='\t')
pval = pval.loc[pval['qvalue']<=0.05]  ##eSTRs at 10% FDR
print(pval.shape[0],' eSTRs at 10% FDR')

#Anova output 
hh = pd.read_csv("/storage/szfeupe/Runs/GTEx_estr/Analysis_by_Tissue/"+Tissue+"/HH/Anova_wg_old.csv", sep=',')
hh1 = hh.loc[hh['gene'].isin(pval['gene'])] 
print(hh1.shape[0], ' eSTRs and their anova p-values')

hh1['st1']=[0]*len(hh1)
hh1['st2']=[0]*len(hh1)
hh1['st1'][hh1['delta_aic']>0] = 1
hh1['st2'][hh1['delta_bic']>0] = 2
hh1['strprefer']=hh1['st1']+hh1['st2']


hh1.to_csv("/storage/szfeupe/Runs/GTEx_estr/Analysis_by_Tissue/"+Tissue+"/HH/ESTRs_Anova", sep='\t', index=False)

hh1

In [ ]:
print(OUT3.loc[(OUT3['Blood_eSTR?']==1) & (OUT3['eSTR_In_tissues']==7)].shape)
print(OUT3.loc[(OUT3['AdipSub_eSTR?']==1) & (OUT3['eSTR_In_tissues']==7)].shape)
print(OUT3.loc[(OUT3['ArteryT_eSTR?']==1) & (OUT3['eSTR_In_tissues']>=5)].shape)
print(OUT3.loc[(OUT3['Esophagus_eSTR?']==1) & (OUT3['eSTR_In_tissues']>=5)].shape)
print(OUT3.loc[(OUT3['Fibroblast_eSTR?']==1) & (OUT3['eSTR_In_tissues']>=5)].shape)
print(OUT3.loc[(OUT3['Lung_eSTR?']==1) & (OUT3['eSTR_In_tissues']>=5)].shape)
print(OUT3.loc[(OUT3['MuscleS_eSTR?']==1) & (OUT3['eSTR_In_tissues']>=5)].shape)

In [ ]:

#wholeblood
wb = pd.read_csv('/storage/szfeupe/Runs/GTEx_estr/Analysis_by_Tissue/WholeBlood/Lin_Reg_OutFin.txt', sep='\t')
wb['ID']=wb['gene']+wb['str.id']
wb = wb[col]; wb.columns=['chrom','str.start','str.id','gene','ID','beta_blood','blood_pval']
#adipose subcutaneous
ads= pd.read_csv('/storage/szfeupe/Runs/GTEx_estr/Analysis_by_Tissue/Adipose-Subcutaneous/Lin_Reg_OutFin.txt', sep='\t')
ads['ID']=ads['gene']+ads['str.id']
ads = ads[col]; ads.columns=['chrom','str.start','str.id','gene','ID','beta_AdipSub','AdipSub_pval']
#cell T Fibroblast
ctf= pd.read_csv('/storage/szfeupe/Runs/GTEx_estr/Analysis_by_Tissue/Cells-Transformedfibroblasts/Lin_Reg_OutFin.txt', sep='\t')
ctf['ID']=ctf['gene']+ctf['str.id']
ctf = ctf[col]; ctf.columns=['chrom','str.start','str.id','gene','ID','beta_Fibroblast','Fibroblast_pval']
#Esophagus mucosa
esm= pd.read_csv('/storage/szfeupe/Runs/GTEx_estr/Analysis_by_Tissue/Esophagus-Mucosa/Lin_Reg_OutFin.txt', sep='\t')
esm['ID']=esm['gene']+esm['str.id']
esm = esm[col]; esm.columns=['chrom','str.start','str.id','gene','ID','beta_Esophag','Esophag_pval']
#Muscle Skeletal
ms = pd.read_csv('/storage/szfeupe/Runs/GTEx_estr/Analysis_by_Tissue/Muscle-Skeletal/Lin_Reg_OutFin.txt', sep='\t')
ms['ID']=ms['gene']+ms['str.id']
ms = ms[col] ; ms.columns=['chrom','str.start','str.id','gene','ID','beta_MuscleS','MuscleS_pval']
#Lung
lng= pd.read_csv('/storage/szfeupe/Runs/GTEx_estr/Analysis_by_Tissue/Lung/Lin_Reg_OutFin.txt', sep='\t')
lng['ID']=lng['gene']+lng['str.id']
lng = lng[col]; lng.columns=['chrom','str.start','str.id','gene','ID','beta_Lung','Lung_pval']
#Arterial Tibial
art= pd.read_csv('/storage/szfeupe/Runs/GTEx_estr/Analysis_by_Tissue/Artery-Tibial/Lin_Reg_OutFin.txt', sep='\t')
art['ID']=art['gene']+art['str.id']
art = art[col]; art.columns=['chrom','str.start','str.id','gene','ID','beta_ArteryT','ArteryT_pval']
rows=['chrom','str.start','str.id','gene','ID']
OUT=pd.concat([wb[rows],ads[rows],ctf[rows],esm[rows],ms[rows],lng[rows],art[rows]]).drop_duplicates().reset_index(drop=True)
print(wb.shape[0], ads.shape[0],ctf.shape[0], esm.shape[0],ms.shape[0],lng.shape[0],art.shape[0])
#
#FDR_correction
#wbp['Blood_eSTR?'] = np.where(wbp['qvalue']<=0.1, 1, 0)

wbp = pd.read_csv('/storage/szfeupe/Runs/GTEx_estr/Analysis_by_Tissue/WholeBlood/PQValues.txt', sep='\t')
wbp['Blood_eSTR?']= np.where(wbp['qvalue']<=0.1, 1, 0); wbp['ID']=wbp['gene']+wbp['str.id'] ;wbp['blood_qval']=wbp['qvalue']
Col=['chrom','str.start','str.id','gene','ID', 'blood_qval', 'Blood_eSTR?']; wbp=wbp[Col]
print('Whole blood',wbp.shape[0])
adsp= pd.read_csv('/storage/szfeupe/Runs/GTEx_estr/Analysis_by_Tissue/Adipose-Subcutaneous/PQValues.txt', sep='\t')
adsp['AdipSub_eSTR?']= np.where(adsp['qvalue']<=0.1, 1, 0);  adsp['ID']= adsp['gene']+ adsp['str.id']; adsp['AdipSub_qval']=  adsp['qvalue']
Col=['chrom','str.start','str.id','gene','ID', 'AdipSub_qval', 'AdipSub_eSTR?']; adsp=adsp[Col]
print('Adipose Subcutanous',adsp.shape[0])
ctfp= pd.read_csv('/storage/szfeupe/Runs/GTEx_estr/Analysis_by_Tissue/Cells-Transformedfibroblasts/PQValues.txt', sep='\t')
ctfp['Fibroblast_eSTR?']= np.where(ctfp['qvalue']<=0.1, 1, 0);  ctfp['ID']= ctfp['gene']+ ctfp['str.id']; ctfp['Fibroblast_qval']=  ctfp['qvalue']
Col=['chrom','str.start','str.id','gene','ID', 'Fibroblast_qval', 'Fibroblast_eSTR?']; ctfp= ctfp[Col]
print('Cell Fibroblast',ctfp.shape[0])
esmp= pd.read_csv('/storage/szfeupe/Runs/GTEx_estr/Analysis_by_Tissue/Esophagus-Mucosa/PQValues.txt', sep='\t')
esmp['Esophagus_eSTR?']= np.where(esmp['qvalue']<=0.1, 1, 0); esmp['ID']= esmp['gene']+ esmp['str.id']; esmp['Esophagus_qval']=  esmp['qvalue']
Col=['chrom','str.start','str.id','gene','ID', 'Esophagus_qval', 'Esophagus_eSTR?']; esmp = esmp[Col]
print('Esophagous Mucosa',esmp.shape[0])
msp = pd.read_csv('/storage/szfeupe/Runs/GTEx_estr/Analysis_by_Tissue/Muscle-Skeletal/PQValues.txt', sep='\t')
msp['MuscleS_eSTR?']= np.where(msp['qvalue']<=0.1, 1, 0);  msp['ID']=  msp['gene']+  msp['str.id'];  msp['MuscleS_qval']= msp['qvalue']
Col=['chrom','str.start','str.id','gene','ID', 'MuscleS_qval', 'MuscleS_eSTR?']; msp = msp[Col]
print('Muscle skeletal',msp.shape[0])
lngp= pd.read_csv('/storage/szfeupe/Runs/GTEx_estr/Analysis_by_Tissue/Lung/PQValue.tsv', sep='\t')
lngp['Lung_eSTR?']= np.where(lngp['qvalue']<=0.1, 1, 0);   lngp['ID']=  lngp['gene']+  lngp['str.id'];  lngp['Lung_qval']= lngp['qvalue']
Col=['chrom','str.start','str.id','gene','ID', 'Lung_qval', 'Lung_eSTR?']; lngp = lngp[Col]
print('Lungs', lngp.shape[0])
artp= pd.read_csv('/storage/szfeupe/Runs/GTEx_estr/Analysis_by_Tissue/Artery-Tibial/PQValues.txt', sep='\t')
artp['ArteryT_eSTR?']=  np.where(artp['qvalue']<=0.1, 1, 0); artp['ID']= artp['gene']+ artp['str.id']; artp['ArteryT_qval']= artp['qvalue']
Col=['chrom','str.start','str.id','gene','ID', 'ArteryT_qval', 'ArteryT_eSTR?'];  artp= artp[Col]
print('Artery tibial', artp.shape[0])
OUT['str.start'] = OUT['str.start'].astype(int)
OUT.shape